In [ ]:
# Default parameters

systmpfs = '/tmp'
num_workers = 32
budget = 512
algorithm = 'PSO'
timeout = 5
period = 'W'
mean = -2.0e3
var = 1.5e3

In [ ]:
import os
import tempfile
import subprocess

from concurrent import futures

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
sns.set(rc={'figure.figsize': (15, 10), 'figure.dpi' : 250})

import nevergrad as ng
import geotopy as gtp

In [ ]:
class GEOtopCalibrationRun(gtp.GEOtop):
    
    def __init__(self, *args, **kwargs):
        
        super().__init__(*args, **kwargs)
        self.dz=np.array([25.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0, 50.0])


    def preprocess(self, working_dir, *args, **kwargs):
        
        soil_path = os.path.join(working_dir, 'soil0001.txt')
        soil = pd.DataFrame({'dz': self.dz, 'init psi': args[0]})
        soil.to_csv(soil_path, index=False)
        

    def postprocess(self, working_dir):
        
        ice_path = os.path.join(working_dir, 'output/ice0001.txt')
        ice = pd.read_csv(ice_path, 
                          usecols=[0, 7], 
                          header=0, 
                          names=["date", "theta"], 
                          na_values=['-9999'], 
                          parse_dates=True,
                          infer_datetime_format=True,
                          index_col=0)
        
        liq_path = os.path.join(working_dir, 'output/liq0001.txt')
        liq = pd.read_csv(liq_path, 
                          usecols=[0, 7], 
                          header=0, 
                          names=["date", "theta"], 
                          na_values=['-9999'], 
                          parse_dates=True, 
                          infer_datetime_format=True,
                          index_col=0)
        
        sim = ice + liq
        
        obs_path = os.path.join(working_dir, 'obs.csv')
        obs = pd.read_csv(obs_path, 
                          usecols=[0, 7], 
                          header=0, 
                          names=["date", "theta"], 
                          na_values=['-9999'], 
                          parse_dates=True, 
                          infer_datetime_format=True,
                          index_col=0)
        obs = obs
        
        return sim, obs

    
def diff_metric(obs, sim):
    
    diff = sim - obs
    weekly_diff = diff.resample(period).mean()
            
    return np.abs(weekly_diff['theta'].mean())

In [ ]:
def compare(observations, simulation, periods=None, name=None, unit=None, cum=False, rel=False):
    
    if not periods:
        periods = {'Daily': 'D', 'Weekly': 'W', 'Monthly': 'M'}
    
    fig, axes = plt.subplots(ncols=3, 
                             nrows=len(periods), 
                             constrained_layout=True)
    
    if name:
        fig.suptitle(name)
    
    for i, (Tstr, T) in enumerate(periods.items()):
        comp_plot, diff_plot, hist_plot = axes[i, :]
        
        if cum:
            obs_resampled = observations.resample(T).sum()
            sim_resampled = simulation.resample(T).sum()
        else:
            obs_resampled = observations.resample(T).mean()
            sim_resampled = simulation.resample(T).mean()
        
        err = obs_resampled - sim_resampled        
        if rel:
            err = err / obs_resampled.abs()
        
        data = pd.DataFrame({'Observations': obs_resampled, 'Simulation': sim_resampled})
        sns.lineplot(data=data, ax=comp_plot)
        comp_plot.set_title(Tstr)
        comp_plot.set_xlabel("")
        if unit:
            comp_plot.set_ylabel(f'[{unit}]')
        
        sns.lineplot(data=err, ax=diff_plot)
        plt.setp(diff_plot.get_xticklabels(), rotation=20)
        if rel:
            diff_plot.set_ylabel(f'Relative error')
            diff_plot.yaxis.set_major_formatter(mtick.PercentFormatter(xmax=1.0))
        elif unit:
            diff_plot.set_ylabel(f'Error [{unit}]')
        else:
            diff_plot.set_ylabel(f'Error')
        
        sns.distplot(err, rug=True, vertical=True, hist=True, ax=hist_plot)
        y1, y2 = diff_plot.get_ylim()
        hist_plot.set_ylim(y1,y2)
        hist_plot.set_yticklabels([])
        hist_plot.set_ylabel("")
    return fig

In [ ]:
with tempfile.TemporaryDirectory() as tmpdir:
    
    model = GEOtopCalibrationRun('data/inputs', exe='../../geotop/build/geotop')
    sim, obs = model.eval(np.full(10,mean), working_dir=tmpdir)
    loss = diff_metric(sim, obs)
    
    print(f"Default initialization (all {mean})")
    print(f"Before optimization loss is {loss}")

    compare(obs["theta"], sim["theta"], name='Soil moisture content @ 50')
    plt.show()

In [ ]:
def loss_function(x):
    
    model = GEOtopCalibrationRun('data/inputs', 
                                 exe='../../geotop/build/geotop',
                                 run_args={'check': True, 'capture_output': True, 'timeout': timeout})
    
    with tempfile.TemporaryDirectory(dir=systmpfs) as tmpdir:
        try:
            sim, obs = model.eval(mean + var * x , working_dir=tmpdir)
        except subprocess.CalledProcessError:
            return np.Inf
        except subprocess.TimeoutExpired:
            return np.Inf
    
    return diff_metric(obs, sim)

optimizer = ng.optimizers.registry[algorithm](parametrization=10, budget=budget, num_workers=num_workers)

with futures.ProcessPoolExecutor(max_workers=optimizer.num_workers) as executor:
    recommendation = optimizer.minimize(loss_function, executor=executor, batch_mode=False)

In [ ]:
with tempfile.TemporaryDirectory() as tmpdir:
    
    print(f"Recommended value is {recommendation.value}")
    print(f"After optimization loss is {recommendation.loss}")
    
    model = GEOtopCalibrationRun('data/inputs', exe='../../geotop/build/geotop')
    sim, obs = model.eval(mean + var * recommendation.value, working_dir=tmpdir)
    loss = diff_metric(sim, obs)

    compare(obs["theta"], sim["theta"], name='Soil moisture content @ 50')
    plt.show()